In [164]:
import pandas as pd
import json
import pprint
import numpy as np
import os

import string
import unicodedata
import re

import matplotlib.pyplot as plt 

from nltk.sentiment.vader import SentimentIntensityAnalyzer
#analyser = SentimentIntensityAnalyzer()

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

from gensim import corpora, models
from keras import layers, models, optimizers
from sklearn.decomposition import LatentDirichletAllocation
from yellowbrick.classifier import ClassificationReport

pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

#preprocessing pipeline
#Pipeline models features like word count, tfidf, word density, word embeddings (GloVe)

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from keras.preprocessing import text, sequence
import xgboost




In [165]:
hs = pd.read_csv('hatespeech.csv', encoding="ISO-8859-1",index_col=6, keep_default_na=False)
#print(hs.head())

orig = pd.read_csv('NAACL_SRW_2016.csv', index_col=0, header=None)
orig.index.name = 'ID'
orig = orig.rename(columns={1: 'Class'})
orig.index = orig.index.astype(str)
#print(orig.head())

#merging the two dataframes
hs = pd.merge(hs, orig, how='inner', left_index=True, right_index=True)
#print(hs.head())
df = hs

In [166]:
df.head()

,Unnamed: 0,Authors,Descriptions,Favorite Count,Follower Count,Hashtags,Locations,Time Tweeted,Tweets,User Mentions,Class
ID,,,,,,,,,,,
319399851215433729,2004,CreatrixKali,Literary Creatrix for Alternative SpiritMag. Composer of articles of quirk and sometimes even remotely interesting status updates. And I'm in a book! Link below,0,169,"[{'text': 'MKR', 'indices': [37, 41]}]",Australia,2013-04-03 10:43:53,Oh yeah Colin! Smash those girls! :D #MKR,[],sexism
320817818222358529,5148,quincepoacher,"AKA Queenotisblue. Sociology, politics, policy, food, Gen Xer, Coburger & grumpy old lady. Social justice, education & employment stuff.",0,425,"[{'text': 'MKR', 'indices': [68, 72]}]",Melbourne,2013-04-07 08:38:23,It's insane they keep bringing people back. When will this show end #MKR,[],none
324114200450437120,5167,MarkTramby,"Mad keen Manly Sea Eagles supporter, love red wine (the better stuff), cricket ( all forms) and my wonderful family",0,330,"[{'text': 'MKR', 'indices': [16, 20]}, {'text': 'TheVoiceAu', 'indices': [92, 103]}]",Brisbane,2013-04-16 10:57:01,"@berkeley_eagle #MKR this shit show has more comebacks than Johnny Farnham, ok back to the #TheVoiceAu","[{'screen_name': 'berkeley_eagle', 'name': 'john G', 'id': 33840083, 'id_str': '33840083', 'indices': [0, 15]}]",none
326286656854454273,5221,BinnyD,"Reclusive nillionaire Whovian who enjoys television, arts, crafts and poking dead things with a stick.",0,211,"[{'text': 'MKR', 'indices': [18, 22]}]","NSW, Australia",2013-04-22 10:49:35,*sigh* oh Colin ? #MKR,[],none
381988216292655104,3612,YesYoureSexist,Inspired by @YesYoureRacist.,0,24041,[],,2013-09-23 03:47:42,"RT @brian_day15: I swear, I'm not sexist, but I honestly just cannot stand the woman college football announcer on ESPN2","[{'screen_name': 'brian_day15', 'name': 'Brian Day', 'id': 392181216, 'id_str': '392181216', 'indices': [3, 15]}]",sexism


In [167]:
df_short = df[['Tweets','Class']]
# df = df.dropna(subset=['Label'])
df_short = df_short.dropna()
df_short.columns = ['data','label']

In [168]:
df_short.head()

,data,label
ID,,
319399851215433729,Oh yeah Colin! Smash those girls! :D #MKR,sexism
320817818222358529,It's insane they keep bringing people back. When will this show end #MKR,none
324114200450437120,"@berkeley_eagle #MKR this shit show has more comebacks than Johnny Farnham, ok back to the #TheVoiceAu",none
326286656854454273,*sigh* oh Colin ? #MKR,none
381988216292655104,"RT @brian_day15: I swear, I'm not sexist, but I honestly just cannot stand the woman college football announcer on ESPN2",sexism


In [169]:
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

#code source is http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "i'm": "i am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expand_contractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    text = c_re.sub(replace, text.lower())
    return text

def strip_mentions_hashtags(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

def high_precision(text):
    for token in text:
        words = text.split()
        assert words.isLower() == word.isUpper()
        
        

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    #text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


In [170]:
df = df_short

# data cleaning and processing
df['data'] = df['data'].apply(lambda x : strip_links(x))
df['data'] = df['data'].apply(lambda x : expand_contractions(x))
df['data'] = df['data'].apply(lambda x : strip_mentions_hashtags(x))
df['data'] = df['data'].apply(lambda x : remove_special_characters(x))

In [196]:
df.to_csv('hatespeech_preporcess.csv')

In [172]:
df = df.drop_duplicates()
df = df.dropna()

In [173]:
#splitting dataset into train and validation set (80% - 20% default)

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['data'], df['label'],)


#Label Encoder converts yes and no into 1 and 0

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

#Count vectorizer will calculate count of every word in text data and will ignore number and whitespaces

count_vector = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vector.fit(df['data'])

#using the count vector defined above, we'll transform our existing text data into train_x_count where every row will indicate 
#tweet and every column will represent count of word indexed at that loc

train_x_count = count_vector.transform(train_x)
valid_x_count = count_vector.transform(valid_x)

#word level tf-idf

tfidf_vect = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['data'])
xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)


#ngram level tf-idf
tfidf_ngram = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', ngram_range=(2,3) ,max_features=5000)
tfidf_ngram.fit(df['data'])
xtrain_tfidf_ngram = tfidf_ngram.transform(train_x)
xvalid_tfidf_ngram = tfidf_ngram.transform(valid_x)

from keras.preprocessing import text, sequence
token = text.Tokenizer()
token.fit_on_texts(df['data'])
word_index = token.word_index

train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

In [174]:
df.label.value_counts()

none      10624
sexism    3103 
racism    1921 
Name: label, dtype: int64

In [181]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


tvec = TfidfVectorizer(stop_words=None, max_features=10000, ngram_range=(1, 3))
lr = LogisticRegression()

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score

def lr_cv(splits, X, Y, pipeline, average_method):
    
    kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=777)
    accuracy = []
    precision = []
    recall = []
    f1 = []
    for train, test in kfold.split(X, Y):
        lr_fit = pipeline.fit(X[train], Y[train])
        prediction = lr_fit.predict(X[test])
        scores = lr_fit.score(X[test],Y[test])
        
        accuracy.append(scores * 100)
        precision.append(precision_score(Y[test], prediction, average=average_method)*100)
        print('              None    Sexism     Racism')
        print('precision:',precision_score(Y[test], prediction, average=None))
        recall.append(recall_score(Y[test], prediction, average=average_method)*100)
        print('recall:   ',recall_score(Y[test], prediction, average=None))
        f1.append(f1_score(Y[test], prediction, average=average_method)*100)
        print('f1 score: ',f1_score(Y[test], prediction, average=None))
        print('-'*50)

    print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
    print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
    print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
    print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))

In [182]:
from sklearn.pipeline import Pipeline
original_pipeline = Pipeline([
    ('vectorizer', tvec),
    ('classifier', lr)
])
lr_cv(5, df.data, df.label, original_pipeline, 'macro')

              None    Sexism     Racism
('precision:', array([0.80196464, 0.82539683, 0.86826347]))
('recall:   ', array([0.96047059, 0.54025974, 0.46698873]))
('f1 score: ', array([0.87408994, 0.65306122, 0.60732984]))
--------------------------------------------------
              None    Sexism     Racism
('precision:', array([0.79290172, 0.8313253 , 0.84227129]))
('recall:   ', array([0.95670588, 0.5390625 , 0.42995169]))
('f1 score: ', array([0.86713585, 0.65402844, 0.56929638]))
--------------------------------------------------
              None    Sexism     Racism
('precision:', array([0.79441427, 0.83739837, 0.86928105]))
('recall:   ', array([0.96376471, 0.53645833, 0.42834138]))
('f1 score: ', array([0.87093345, 0.65396825, 0.57389428]))
--------------------------------------------------
              None    Sexism     Racism
('precision:', array([0.791423  , 0.8372093 , 0.82026144]))
('recall:   ', array([0.95529412, 0.5625    , 0.40483871]))
('f1 score: ', array([0.865

In [ ]:
from imblearn.pipeline import make_pipeline 

In [191]:
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
ROS_pipeline = make_pipeline(tvec, RandomOverSampler(random_state=777),lr)
SMOTE_pipeline = make_pipeline(tvec, SMOTE(random_state=777),lr)


In [190]:
!pip install imblearn

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
     |████████████████████████████████| 174kB 3.5MB/s eta 0:00:01
  Stored in directory: /Users/i504969/Library/Caches/pip/wheels/94/6c/0c/d7254937a767ff72814aa542997d0e889bed37c1d31ba3de1a
Successfully built imbalanced-learn


In [193]:
lr_cv(5, df.data, df.label, SMOTE_pipeline, 'macro')

              None    Sexism     Racism
('precision:', array([0.87350959, 0.69396552, 0.5799458 ]))
('recall:   ', array([0.79294118, 0.83636364, 0.68921095]))
('f1 score: ', array([0.83127775, 0.75853946, 0.62987491]))
--------------------------------------------------
              None    Sexism     Racism
('precision:', array([0.85075403, 0.64937759, 0.54068966]))
('recall:   ', array([0.76988235, 0.81510417, 0.63123994]))
('f1 score: ', array([0.8083004 , 0.72286374, 0.58246657]))
--------------------------------------------------
              None    Sexism     Racism
('precision:', array([0.86252546, 0.68736142, 0.57202797]))
('recall:   ', array([0.79717647, 0.80729167, 0.65861514]))
('f1 score: ', array([0.82856444, 0.74251497, 0.61227545]))
--------------------------------------------------
              None    Sexism     Racism
('precision:', array([0.86208678, 0.69164882, 0.56060606]))
('recall:   ', array([0.78541176, 0.84114583, 0.65645161]))
('f1 score: ', array([0.821

In [195]:
from imblearn.under_sampling import NearMiss, RandomUnderSampler
RUS_pipeline = make_pipeline(tvec, RandomUnderSampler(random_state=777),lr)
NM1_pipeline = make_pipeline(tvec, NearMiss(ratio='not minority',random_state=777, version = 1),lr)
NM2_pipeline = make_pipeline(tvec, NearMiss(ratio='not minority',random_state=777, version = 2),lr)
# NM3_pipeline = make_pipeline(tvec, NearMiss(ratio=nm3_dict,random_state=777, version = 3, n_neighbors_ver3=4),lr)


lr_cv(5, df.data, df.label, RUS_pipeline, 'macro')


              None    Sexism     Racism
('precision:', array([0.89252078, 0.5877193 , 0.61243386]))
('recall:   ', array([0.75811765, 0.87012987, 0.74557166]))
('f1 score: ', array([0.81984733, 0.70157068, 0.6724764 ]))
--------------------------------------------------
              None    Sexism     Racism
('precision:', array([0.8819209 , 0.56065574, 0.568     ]))
('recall:   ', array([0.73458824, 0.890625  , 0.68599034]))
('f1 score: ', array([0.80154044, 0.68812877, 0.6214442 ]))
--------------------------------------------------
              None    Sexism     Racism
('precision:', array([0.88292683, 0.58185053, 0.60580913]))
('recall:   ', array([0.76658824, 0.8515625 , 0.70531401]))
('f1 score: ', array([0.82065491, 0.69133192, 0.65178571]))
--------------------------------------------------
              None    Sexism     Racism
('precision:', array([0.87111362, 0.56198347, 0.55761589]))
('recall:   ', array([0.72517647, 0.88541667, 0.67903226]))
('f1 score: ', array([0.791

In [10]:
#splitting dataset into train and validation set (80% - 20% default)

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['data'], df['label'],)


#Label Encoder converts yes and no into 1 and 0

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

#Count vectorizer will calculate count of every word in text data and will ignore number and whitespaces

count_vector = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vector.fit(df['data'])

#using the count vector defined above, we'll transform our existing text data into train_x_count where every row will indicate 
#tweet and every column will represent count of word indexed at that loc

train_x_count = count_vector.transform(train_x)
valid_x_count = count_vector.transform(valid_x)

#word level tf-idf

tfidf_vect = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['data'])
xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)


#ngram level tf-idf
tfidf_ngram = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', ngram_range=(2,3) ,max_features=5000)
tfidf_ngram.fit(df['data'])
xtrain_tfidf_ngram = tfidf_ngram.transform(train_x)
xvalid_tfidf_ngram = tfidf_ngram.transform(valid_x)

from keras.preprocessing import text, sequence
token = text.Tokenizer()
token.fit_on_texts(df['data'])
word_index = token.word_index

train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

In [58]:
#splitting dataset into train and validation set (80% - 20% default)

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['data'], df['label'])


#Label Encoder converts yes and no into 1 and 0

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

#Count vectorizer will calculate count of every word in text data and will ignore number and whitespaces

count_vector = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vector.fit(df['data'])



CountVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='\\w{1,}', tokenizer=None,
        vocabulary=None)

In [59]:
train_x_count = count_vector.transform(train_x)
valid_x_count = count_vector.transform(valid_x)



In [62]:
train_x_count.shape

(11736, 15460)

In [63]:
#word level tf-idf

tfidf_vect = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['data'])
xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)



In [64]:
xtrain_tfidf

<11736x5000 sparse matrix of type '<type 'numpy.float64'>'
	with 147214 stored elements in Compressed Sparse Row format>

In [39]:
df_very_short

,data,label
ID,,
319399851215433729,oh yeah colin smash those girls d,sexism
320817818222358529,it is insane they keep bringing people back when will this show end,none


In [22]:
count_vector = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}')

In [40]:
train_x_count = count_vector.fit(df_very_short['data'])

In [52]:
x=count_vector.transform(train_x)

In [57]:
count_vector.get_feature_names()

[u'back',
 u'bringing',
 u'colin',
 u'd',
 u'end',
 u'girls',
 u'insane',
 u'is',
 u'it',
 u'keep',
 u'oh',
 u'people',
 u'show',
 u'smash',
 u'they',
 u'this',
 u'those',
 u'when',
 u'will',
 u'yeah']

In [51]:
train_x

dtype('O')

In [55]:
x.shape

(11736, 20)

In [56]:
train_x.shape

(11736,)

In [77]:
import gensim
# ata = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);

lda_model = gensim.models.LdaMulticore(df['data'], num_topics=10, id2word=train_x, passes=2, workers=2)

TypeError: unsupported operand type(s) for +: 'int' and 'unicode'

In [86]:
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(train_x_count)
topic_word = lda_model.components_ 
vocab = count_vector.get_feature_names()

# view the topic models
n_top_words = 20
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [87]:
topic_words

array([u'pink', u'putting', u'walking', u'alive', u'catwalk', u'easily',
       u'ways', u'forcing', u'animal', u'expected', u'pants', u'burned',
       u'aisha', u'values', u'apply', u'slut', u'sauce', u'streets',
       u'satan', u'liars'], dtype='<U78')

In [91]:
df_sexist_tweets = df[df["label"]=='sexism']

In [93]:
df_sexist_tweets.head()

,data,label
ID,,
319399851215433729,oh yeah colin smash those girls d,sexism
381988216292655104,rt day15 i swear i am not sexist but i honestly just cannot stand the woman college football announcer on espn2,sexism
381989237072990208,rt call me sexist but i think some women are seriously lacking knowledge l when it comes to feminism,sexism
381989256404533251,rt call me sexist but females really need to stop acting like they are big football fans just stop,sexism
381989513691529216,rt the mine is no place for a woman to work call me sexist but a woman as a rock driller is not practical,sexism


In [100]:
train_x_sexist, valid_x_sexist, train_y_sexist, valid_y_sexist = model_selection.train_test_split(df_sexist_tweets['data'], df_sexist_tweets['label'])


count_vector = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vector.fit(df_sexist_tweets['data'])

#using the count vector defined above, we'll transform our existing text data into train_x_count where every row will indicate 
#tweet and every column will represent count of word indexed at that loc

train_x_count_sexist = count_vector.transform(train_x_sexist)
valid_x_count_sexist = count_vector.transform(valid_x_sexist)


In [101]:
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(train_x_count_sexist)
topic_word = lda_model.components_ 
vocab = count_vector.get_feature_names()

# view the topic models
n_top_words = 20
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [103]:
topic_summaries

[u'says cunt history means lie attractive truth nothing ladies vagina pussy manu beautiful well six arm ahem sucking sign fail',
 u'kat bitch oh male even thing hope fucking everyone best god think last tell feminism models got like rt point',
 u'someone nice mom head give celine sister roads uh account cleaning across followers humor founder money top examples elephant synonym',
 u'promo yes joke run ash repeat tennis putting engage k shes found trolling enough evidently whos 93 students heads setting',
 u'one saw go trying tonight colin friend death becoming halim topical chrisdowns poor sudden please 33 meat board tight mistake',
 u'nikki katie people 3 rt child must put either years choice able equality thought alone rules 1 sick honestly 2',
 u'new day talking exactly tart full deconstructed u econ sausage second cock shit anyway weeks feed properly steve dishes hateful',
 u'ever year else explain hard deserve idea needs seen abuse gets get believe human first round competition wa

In [98]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 


stop_words = set(stopwords.words('english')) 

def stop_word_removal(text):
    text_tokens = word_tokenize(text)
    filtered_sentence = [w for w in text_tokens if not w in stop_words] 
    return ' '.join(filtered_sentence)

df_sexist_tweets['data'] = df_sexist_tweets['data'].apply(lambda x : stop_word_removal(x))
  
    
    

This sample sentence , showing stop words filtration .


In [99]:
df_sexist_tweets['data'] = df_sexist_tweets['data'].apply(lambda x : stop_word_removal(x))


/Users/i504969/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [104]:
df_racism_tweets = df[df["label"]=='racism']

train_x_racism, valid_x_racism, train_y_racism, valid_y_racism = model_selection.train_test_split(df_racism_tweets['data'], df_racism_tweets['label'])


count_vector = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vector.fit(df_racism_tweets['data'])

#using the count vector defined above, we'll transform our existing text data into train_x_count where every row will indicate 
#tweet and every column will represent count of word indexed at that loc

train_x_count_racism = count_vector.transform(train_x_racism)
valid_x_count_racsim = count_vector.transform(valid_x_racism)


lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(train_x_count_racism)
topic_word = lda_model.components_ 
vocab = count_vector.get_feature_names()

# view the topic models
n_top_words = 20
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [106]:
topic_words

array([u'arab', u'liars', u'justice', u'self', u'occupied', u'serving',
       u'genociadal', u'araujog', u'exterminated', u'to', u'hate', u'of',
       u'minorities', u'people', u'that', u'every', u'democracy',
       u'muslims', u'freedom', u'me'], dtype='<U40')

In [ ]:
import tweepy #https://github.com/tweepy/tweepy
import csv

#Twitter API credentials
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""


def get_all_tweets(screen_name):
	#Twitter only allows access to a users most recent 3240 tweets with this method
	
	#authorize twitter, initialize tweepy
	auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
	auth.set_access_token(access_key, access_secret)
	api = tweepy.API(auth)
	
	#initialize a list to hold all the tweepy Tweets
	alltweets = []	
	
	#make initial request for most recent tweets (200 is the maximum allowed count)
	new_tweets = api.user_timeline(screen_name = screen_name,count=200)
	
	#save most recent tweets
	alltweets.extend(new_tweets)
	
	#save the id of the oldest tweet less one
	oldest = alltweets[-1].id - 1
	
	#keep grabbing tweets until there are no tweets left to grab
	while len(new_tweets) > 0:
		print "getting tweets before %s" % (oldest)
		
		#all subsiquent requests use the max_id param to prevent duplicates
		new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
		
		#save most recent tweets
		alltweets.extend(new_tweets)
		
		#update the id of the oldest tweet less one
		oldest = alltweets[-1].id - 1
		
		print "...%s tweets downloaded so far" % (len(alltweets))
	
	#transform the tweepy tweets into a 2D array that will populate the csv	
	outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]
	
	#write the csv	
	with open('%s_tweets.csv' % screen_name, 'wb') as f:
		writer = csv.writer(f)
		writer.writerow(["id","created_at","text"])
		writer.writerows(outtweets)
	
	pass


if __name__ == '__main__':
	#pass in the username of the account you want to download
	get_all_tweets("J_tsar")

In [163]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
example_sent = "This is a sample sentence, showing off the stop words filtration."
  
stop_words = set(stopwords.words('english')) 
  
word_tokens = word_tokenize(example_sent) 
  
filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
filtered_sentence = [] 

  
for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w)

In [11]:
from __future__ import division
import sys,json,math
import os
import numpy as np


def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

def load_word2vec(filename):
    # Returns a dict containing a {word: numpy array for a dense word vector} mapping.
    # It loads everything into memory.
    
    w2vec={}
    with open(filename,"r") as f_in:
        for line in f_in:
            line_split=line.replace("\n","").split()
            w=line_split[0]
            vec=np.array([float(x) for x in line_split[1:]])
            w2vec[w]=vec
    return w2vec

In [160]:
emb_matrix

NameError: name 'emb_matrix' is not defined

In [12]:
word_to_vec_dict = loadGloveModel('glove.twitter.27B.200d.txt')

import numpy
emb_matrix = numpy.zeros((len(word_index)+1, 200))
for word, i in word_index.items():
    emb_vect = word_to_vec_dict.get(word)
    if emb_vect is not None:
        emb_matrix[i] = emb_vect

Loading Glove Model
('Done.', 1193514, ' words loaded!')


In [13]:
dataset_train = pd.DataFrame(train_x)
dataset_valid = pd.DataFrame(valid_x)


In [14]:
dataset_for_topic_model = pd.concat([dataset_train, dataset_valid], axis=0)


In [15]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dataset_for_topic_model['data'])
lda = LatentDirichletAllocation(random_state=0)
doc_topic_matrix = lda.fit_transform(X)


In [16]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()


In [17]:
dataset_train['comp'] = dataset_train['data'].apply(lambda x : analyser.polarity_scores(x)['compound'])
dataset_train['pos'] = dataset_train['data'].apply(lambda x : analyser.polarity_scores(x)['pos'])
dataset_train['neu'] = dataset_train['data'].apply(lambda x : analyser.polarity_scores(x)['neu'])
dataset_train['neg'] = dataset_train['data'].apply(lambda x : analyser.polarity_scores(x)['neg'])

dataset_valid['comp'] = dataset_valid['data'].apply(lambda x : analyser.polarity_scores(x)['compound'])
dataset_valid['pos'] = dataset_valid['data'].apply(lambda x : analyser.polarity_scores(x)['pos'])
dataset_valid['neu'] = dataset_valid['data'].apply(lambda x : analyser.polarity_scores(x)['neu'])
dataset_valid['neg'] = dataset_valid['data'].apply(lambda x : analyser.polarity_scores(x)['neg'])

In [18]:
# combine sentiment features with count vectorizer features
train_x_count = pd.DataFrame(np.concatenate((train_x_count.toarray(), dataset_train[['comp', 'pos', 'neu', 'neg']].values, doc_topic_matrix[:len(train_x)]), axis=1))
valid_x_count = pd.DataFrame(np.concatenate((valid_x_count.toarray(), dataset_valid[['comp', 'pos', 'neu', 'neg']].values, doc_topic_matrix[len(train_x):]), axis=1))

# combine sentiment features with tf-idf features
xtrain_tfidf = pd.DataFrame(np.concatenate((xtrain_tfidf.toarray(), dataset_train[['comp', 'pos', 'neu', 'neg']].values, doc_topic_matrix[:len(train_x)]), axis=1))
xvalid_tfidf = pd.DataFrame(np.concatenate((xvalid_tfidf.toarray(), dataset_valid[['comp', 'pos', 'neu', 'neg']].values, doc_topic_matrix[len(train_x):]), axis=1))

In [19]:
from sklearn.metrics import f1_score

def train_model(classifier, feature_vector_train,label, feature_vector_valid, is_neural_net = False):
    classifier.fit(feature_vector_train, label)
    prediction = classifier.predict(feature_vector_valid)
    from sklearn.metrics import confusion_matrix
    if not is_neural_net:
        print(confusion_matrix(valid_y, prediction))
    if is_neural_net:
        prediction = prediction.argmax(axis=-1)
    return metrics.accuracy_score(prediction, valid_y), f1_score(prediction, valid_y,average=None)


In [20]:
count_vector = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vector.fit(df['data'])
nb_train_x_count = count_vector.transform(train_x)
nb_valid_x_count = count_vector.transform(valid_x)
tfidf_vect = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['data'])
nb_xtrain_tfidf = tfidf_vect.transform(train_x)
nb_xvalid_tfidf = tfidf_vect.transform(valid_x)



clf = naive_bayes.MultinomialNB()
accuracy, f1 = train_model(clf, nb_train_x_count, train_y, nb_valid_x_count)
print("Naive Bayes (Count Vectors)",accuracy, f1)
accuracy, f1 = train_model(clf, nb_xtrain_tfidf, train_y, nb_xvalid_tfidf)
print("Naive Bayes (TDIDF)", accuracy, f1)

[[2307  169  153]
 [ 135  346    3]
 [ 345    4  450]]
('Naive Bayes (Count Vectors)', 0.793200408997955, array([0.85192024, 0.68993021, 0.6405694 ]))
[[2568   36   25]
 [ 294  190    0]
 [ 551    0  248]]
('Naive Bayes (TDIDF)', 0.7684049079754601, array([0.85004965, 0.53521127, 0.46268657]))


In [33]:
clf = linear_model.LogisticRegression(verbose=1)
accuracy, f1 = train_model(clf, train_x_count, train_y, valid_x_count)
print("LR using CV", accuracy, f1)
accuracy, f1 = train_model(clf, xtrain_tfidf, train_y, xvalid_tfidf)
print("LR using CV", accuracy, f1)

[LibLinear][[2434   86  113]
 [ 149  334    4]
 [ 318    2  472]]
('LR using CV', 0.8282208588957055, array([0.87965305, 0.73487349, 0.68356264]))
[LibLinear][[2495   68   70]
 [ 207  275    5]
 [ 417    0  375]]
('LR using CV', 0.8039366053169734, array([0.86752434, 0.6626506 , 0.60386473]))


In [ ]:
accuracy, f1 = train_model(xgboost.XGBClassifier(verbose=1), train_x_count, train_y, valid_x_count)
print("xgboost using CV", accuracy, f1)
accuracy, f1 = train_model(xgboost.XGBClassifier(verbose=1), xtrain_tfidf, train_y, xvalid_tfidf)
print("xgboost using TFIDF", accuracy, f1)

In [39]:
#Shallow Neural Networks

def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier

In [40]:
classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy, f1 = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)
print ("NN, Ngram Level TF IDF Vectors",  accuracy)

Epoch 1/20
11736/11736 [==============================] - 5s 419us/step - loss: 0.4249
Epoch 2/20
11736/11736 [==============================] - 4s 361us/step - loss: -0.4221
Epoch 3/20
11736/11736 [==============================] - 4s 378us/step - loss: -0.7532
Epoch 4/20
11736/11736 [==============================] - 4s 343us/step - loss: -0.9797
Epoch 5/20
11736/11736 [==============================] - 4s 339us/step - loss: -1.1788
Epoch 6/20
11736/11736 [==============================] - 4s 336us/step - loss: -1.3251
Epoch 7/20
11736/11736 [==============================] - 4s 338us/step - loss: -1.4424
Epoch 8/20
11736/11736 [==============================] - 4s 339us/step - loss: -1.5396
Epoch 9/20
11736/11736 [==============================] - 4s 338us/step - loss: -1.6192
Epoch 10/20
11736/11736 [==============================] - 4s 337us/step - loss: -1.6858
Epoch 11/20
11736/11736 [==============================] - 4s 337us/step - loss: -1.7446
Epoch 12/20
11736/11736 [======

In [44]:
classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy, f1 = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)
print ("NN, Ngram Level TF IDF Vectors",  accuracy)

Epoch 1/2
11736/11736 [==============================] - 5s 415us/step - loss: 0.4216
Epoch 2/2
11736/11736 [==============================] - 4s 339us/step - loss: -0.4022
('NN, Ngram Level TF IDF Vectors', 0.6730572597137015)


In [46]:
def create_rnn_gru():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 200, weights=[emb_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_gru()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print(accuracy)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch 1/2
11736/11736 [==============================] - 56s 5ms/step - loss: 0.0461
Epoch 2/2
11736/11736 [==============================] - 54s 5ms/step - loss: -0.3812
(0.6730572597137015, array([0.80458365, 0.        , 0.        ]))


In [51]:
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical


model = Sequential()

#Add layers to model
model.add(Dense(100, activation='relu', input_shape = (70,)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [52]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

early_stop = EarlyStopping(patience=2) 
#train_seq_x, train_y, valid_seq_x
#fit model
model.fit(train_seq_x, train_y, validation_split=0.3, epochs=20, callbacks=[early_stop])

ValueError: Error when checking target: expected dense_28 to have shape (3,) but got array with shape (1,)

In [54]:
train_y.shape

(11736,)

In [157]:
class Preprop:
    
    def __init__(self):
        self.document = 'document'
    
    def remove_strip_links(self,text):
        link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
        links         = re.findall(link_regex, text)
        for link in links:
            text = text.replace(link[0], ', ')    
        return text
    
    def strip_mentions_hashtags(self, text):
        entity_prefixes = ['@','#']
        for separator in  string.punctuation:
            if separator not in entity_prefixes :
                text = text.replace(separator,' ')
        words = []
        for word in text.split():
            word = word.strip()
            if word:
                if word[0] not in entity_prefixes:
                    words.append(word)
        return ' '.join(words)
    
    def remove_special_characters(text, remove_digits=False):
        pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
        text = re.sub(pattern, '', text)
        #text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        return text
        
    def remove_non_ascii(self, words):
        """Remove non-ASCII characters from list of tokenized words"""
        new_words = []
        for word in words.split():
            word = word.strip()
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        return ' '.join(new_words)

    def to_lowercase(self, words):
        """Convert all characters to lowercase from list of tokenized words"""
        new_words = []
        for word in words.split():
            word = word.strip()
            new_word = word.lower()
            new_words.append(new_word)
        return ' '.join(new_words)

    def remove_punctuation(self, words):
        """Remove punctuation from list of tokenized words"""
        new_words = []
        for word in words.split():
            word = word.strip()
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
        return ' '.join(new_words)

    def replace_numbers(self, words):
        """Replace all interger occurrences in list of tokenized words with textual representation"""
        p = inflect.engine()
        new_words = []
        for word in words.split():
            word = word.strip()
            if word.isdigit():
                new_word = p.number_to_words(word)
                new_words.append(new_word)
            else:
                new_words.append(word)
        return ' '.join(new_words)

    def remove_stopwords(self,words):
        """Remove stop words from list of tokenized words"""
        new_words = []
        for word in words.split():
            word = word.strip()
            if word not in stopwords.words('english'):
                new_words.append(word)
        return ' '.join(new_words)

    def stem_words(self, words):
        """Stem words in list of tokenized words"""
        stemmer = LancasterStemmer()
        stems = []
        for word in words.split():
            word = word.strip()
            stem = stemmer.stem(word)
            stems.append(stem)
        return ' '.join(stems)

    def lemmatize_verbs(self, words):
        """Lemmatize verbs in list of tokenized words"""
        lemmatizer = WordNetLemmatizer()
        lemmas = []
        for word in words.split():
            word = word.strip()
            lemma = lemmatizer.lemmatize(word, pos='v')
            lemmas.append(lemma)
        return ' '.join(lemmas)

    def normalize(self, words):
        words = self.remove_strip_links(words)
        words = self.strip_mentions_hashtags(words)
        words = self.remove_non_ascii(words)
        words = self.to_lowercase(words)
        words = self.remove_punctuation(words)
        words = self.replace_numbers(words)
        words = self.remove_stopwords(words)
        return words



In [155]:
df_new = df


In [147]:
import re, string, unicodedata
import nltk
import contractions
import inflect
import string
import unicodedata
import re
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [158]:
instance = Preprop()
df_new['data'] = df_new['data'].apply(lambda x: instance.normalize(x))

In [159]:
df_new['data']

ID
319399851215433729    oh yeah colin smash girls                                                                                        
320817818222358529    insane keep bringing people back show end                                                                        
324114200450437120    eagle shit show comebacks johnny farnham ok back                                                                 
326286656854454273    sigh oh colin                                                                                                    
381988216292655104    rt day15 swear sexist honestly cannot stand woman college football announcer espn2                               
381989167564996609    science good says men cannot monogamous shit amirite                                                             
381989237072990208    rt call sexist think women seriously lacking knowledge l comes feminism                                          
381989256404533251    rt call sexist females 

In [ ]:
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

#code source is http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "i'm": "i am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expand_contractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    text = c_re.sub(replace, text.lower())
    return text

def strip_mentions_hashtags(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

def high_precision(text):
    for token in text:
        words = text.split()
        assert words.isLower() == word.isUpper()
        
        

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    #text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text



getting tweets before 370943710856355839
...46 tweets downloaded so far


In [109]:
x = 'purvak hi how are you'
x.split()
print x.strip()

purvak hi how are you
